In [1]:
import pandas as pd 
data_df = pd.read_csv('data.csv')
data_df.head()

,question,A,B,C,D,Answer
0,I can't remember _____________ made the teach...,that it was what,what it was that,what was it that,that was it what,B
1,"He let out an ______________ cry, ""we've won!""",excited,exciting,excite,excites,A
2,Is football John's favourite sport? Yes. ____...,Near to,Except,Beside,Next to,D
3,Do you think regular exercise ___________ goo...,benefit from,reach for,make for,go for,C
4,Have you applied ___________ Mr Black _______...,for; to,with; for,with; about,to; for,D


In [2]:
import torch
import sys
import numpy as np
 
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
# Load pre-trained model (weights)
with torch.no_grad():
        model = GPT2LMHeadModel.from_pretrained('gpt2')
        model.eval()
# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
 
def score(sentence):
    tokenize_input = tokenizer.encode(sentence)
    tensor_input = torch.tensor([tokenize_input])
    loss=model(tensor_input, labels=tensor_input)[0]
    return np.exp(loss.detach().numpy())

In [3]:
import numpy as np
from tqdm import tqdm
import re
prediction_list=[]
answer_list= []
for i in tqdm(range(len(data_df))):
    row = data_df.iloc[i]
    question_original = row['question'].strip()
    choice_list = ['A','B','C','D']
    score_list = []
    sentence_list = []

    if ';' in row['A']:
        try:
            for choice in choice_list:
                choice_split = [i.strip() for i in row[choice].split(';')]
                if len(choice_split) == 2:
                    question_text = re.sub('\_+',' '+choice_split[0]+' ', question_original,1)
                    question_text = re.sub('\_+',' '+choice_split[1]+' ', question_text,1)
                if len(choice_split) == 3:
                    question_text = re.sub('\_+',' '+choice_split[0]+' ', question_original,1)
                    question_text = re.sub('\_+',' '+choice_split[1]+' ', question_text,1)
                    question_text = re.sub('\_+',' '+choice_split[2]+' ', question_text,1)
                sentence_list.append(question_text)
        except:
                print(question_original,question_text,choice_split)
                continue
    else:
        sentence_list = [re.sub('\_+',' '+row[choice].strip()+' ', question_original) for choice in choice_list]

    if ('/' in row['A']+row['B']+row['C']+row['D']):
        sentence_list = [re.sub('/','',sentence) for sentence in sentence_list]

    sentence_list = [re.sub(' +',' ', sentence) for sentence in sentence_list]

    for sentence in sentence_list:
        score_list.append(score(sentence))
    final_choice = choice_list[np.argmin(np.array(score_list))]
    answer_list.append(final_choice)
    if final_choice == row['Answer']:
        prediction_list.append(1)
    else:
        prediction_list.append(0)

100%|██████████| 1183/1183 [07:17<00:00,  2.71it/s]


In [4]:
np.array(prediction_list).sum()/len(prediction_list)

0.6686390532544378